In [45]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.special as sc
from haloclustering import statistics as stats
import pickle
from astropy.cosmology import Planck15 as cosmo

In [9]:
pkl_file = '../exclusion-model/exclusion_model_gauss_1h_sampler.pkl'
infile = open(pkl_file, "rb")
sampler = pickle.load(infile)
infile.close()

pkl_file = '../exclusion-model/model_2h_w_beta_sampler.pkl'
infile = open(pkl_file, "rb")
sampler_2h = pickle.load(infile)
infile.close()

(16, 4000)

In [50]:
# get data
df = pd.read_csv('../data/cgm2_casbah_velo_split_table_with_comoving_rho_and_mass.csv')
df['log_rho'] = np.log10(df.rho)
df.sort_values('rho', inplace=True)
df.rho = df.rho / 1000 # convert to Mpc
df = df.query("(HM_0_500 != 'indeterminate') & (7 < mstars < 12)")
df['Hz'] = cosmo.H(df.z).value
vmax = 500.0 # km/s

df.dropna(subset=['mstars'], inplace=True)
massmask = df.eval("8 < mstars < 10.5").values
df.mstars = 10**df.mstars

outcomes = df.HM_0_500
hits = (outcomes.values == "hit") & massmask
misses = (outcomes.values == "miss") & massmask

In [80]:
data_args = (df.rho.values, df.z.values, df.Hz.values, vmax, df.mstars.values, hits, misses)
params_chain = sampler.get_chain(discard=2125, thin=1, flat=True)
params = np.nanmean(params_chain, axis=0)
params_2h_chain = sampler_2h.get_chain(discard=2000, thin=1, flat=True)
params_2h = np.nanmean(params_2h_chain, axis=0)
ll_exc = stats.log_likelihood_exc(*params, *data_args)
ll_2h = stats.log_likelihood_2h(*params_2h, *data_args)

3.603113933775093 1.5462836768996562 0.07666961135741135


In [78]:
bic_exc = stats.bayes_inf_criterion(ll_exc[0], len(params), len(df.z.values))
bic_2h = stats.bayes_inf_criterion(ll_2h[0], len(params_2h), len(df.z.values))

In [79]:
bic_exc, bic_2h

(6821.547515661012, 6805.394688180214)

In [71]:
len(params), len(df.z.values)

(8, 7244)

In [27]:
sampler.get_chain(discard=2125, thin=1, flat=True)

array([[ 3.52562689e+00,  1.71883180e+00,  6.56892246e-02, ...,
         2.38936978e-01,  9.61089478e-01,  3.48741122e+01],
       [ 3.59976179e+00,  1.71502720e+00,  7.84907458e-02, ...,
         2.04809670e-01,  4.13030256e-01,  3.84151874e+01],
       [ 3.78973693e+00,  1.68806365e+00,  2.68689684e-02, ...,
         6.39437884e-02,  9.42900247e-01,  3.52490128e+01],
       ...,
       [ 4.09732459e+00,  1.52549715e+00,  7.26905890e-02, ...,
         1.48024493e-01,  3.62355609e-01,  3.71390945e+01],
       [ 4.20829188e+00,  1.56536364e+00,  6.47986243e-02, ...,
         2.04052218e-01,  1.00251092e+00,  3.29771012e+01],
       [ 4.09361337e+00,  1.59291860e+00,  6.34588633e-02, ...,
         2.53996079e-02, -6.83428088e-02,  4.22705963e+01]])